[View in Colaboratory](https://colab.research.google.com/github/reachnivi/Machine-Learning-Projects/blob/master/Disaster_prediction_Hackerearth.ipynb)

In [0]:
import pandas as pd
import numpy as np


In [0]:
from google.colab import files

In [4]:
!wget --header 'Host: he-s3.s3.amazonaws.com' --user-agent 'Mozilla/5.0 (Windows NT 6.1; rv:61.0) Gecko/20100101 Firefox/61.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-6-1/predict-the-energy-used-612632a9-3f496e7f/a490e594-6-Dataset.zip' --output-document 'a490e594-6-Dataset.zip'

--2018-08-04 15:26:53--  https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-6-1/predict-the-energy-used-612632a9-3f496e7f/a490e594-6-Dataset.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.40.172
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.40.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33194548 (32M) [application/zip]
Saving to: ‘a490e594-6-Dataset.zip’

a490e594-6-Dataset. 100%[===================>]  31.66M  8.01MB/s    in 8.8s    

2018-08-04 15:27:03 (3.58 MB/s) - ‘a490e594-6-Dataset.zip’ saved [33194548/33194548]



In [5]:
!ls

a490e594-6-Dataset.zip	datalab


In [6]:
!unzip a490e594-6-Dataset.zip


Archive:  a490e594-6-Dataset.zip
  inflating: Dataset/Building_Ownership_Use.csv  
  inflating: Dataset/Building_Structure.csv  
  inflating: Dataset/ReadMe.md       
  inflating: Dataset/sample_submission.csv  
  inflating: Dataset/test.csv        
  inflating: Dataset/train.csv       


In [0]:
train = pd.read_csv("Dataset/train.csv")
test = pd.read_csv("Dataset/test.csv")

In [49]:
train.shape

(631761, 14)

In [50]:
train.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id'],
      dtype='object')

In [51]:
test.shape

(421175, 13)

In [103]:
BS = pd.read_csv("Dataset/Building_Structure.csv")
BO = pd.read_csv("Dataset/Building_Ownership_Use.csv")
print(BS.shape)
print(BO.shape)

(1052948, 29)
(1052948, 17)


In [0]:
merged = pd.merge(BS,BO,on="building_id")

In [0]:
train_merged = pd.merge(train,merged,on="building_id",how="left")

In [0]:
test_merged = pd.merge(test,merged,on="building_id",how="left")


In [107]:
train_merged.dtypes.value_counts()

int64      42
object     12
float64     4
dtype: int64

In [0]:
object_columns = train_merged.select_dtypes(include=["object"]).columns

AttributeError: ignored

In [58]:
train_merged[object_columns].head(5)


,area_assesed,building_id,damage_grade,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,condition_post_eq,legal_ownership_status
0,Both,24385bfd2a2,Grade 4,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,Damaged-Not used,Private
1,Both,405d1bbebbf,Grade 2,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Brick/Stone,TImber/Bamboo-Mud,Attached-1 side,Rectangular,Damaged-Repaired and used,Private
2,Both,351d9bc71f6,Grade 1,Steep slope,RC,RCC/RB/RBC,RC,RCC/RB/RBC,Attached-1 side,Rectangular,Not damaged,Private
3,Building removed,2be3a971166,Grade 5,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Timber-Planck,Not attached,Rectangular,Damaged-Rubble clear,Private
4,Both,34c7d073ea6,Grade 3,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,Damaged-Used in risk,Private


In [59]:
train_merged.isnull().sum()

area_assesed                                  0
building_id                                   0
damage_grade                                  0
district_id                                   0
has_geotechnical_risk                         0
has_geotechnical_risk_fault_crack             0
has_geotechnical_risk_flood                   0
has_geotechnical_risk_land_settlement         0
has_geotechnical_risk_landslide               0
has_geotechnical_risk_liquefaction            0
has_geotechnical_risk_other                   0
has_geotechnical_risk_rock_fall               0
has_repair_started                        33417
vdcmun_id                                     0
district_id_x                                 0
vdcmun_id_x                                   0
ward_id_x                                     0
count_floors_pre_eq                           0
count_floors_post_eq                          0
age_building                                  0
plinth_area_sq_ft                       

In [60]:
train_merged['has_repair_started'].value_counts()


0.0    409222
1.0    189122
Name: has_repair_started, dtype: int64

We may have to fill in data in the following ways.


*   If the structure is very less damaged, the repair may not be needed
*   If the structure is severly damaged, the building cannot be repaired
*   If the structure is any where in between, the repair may be required. 

The column may not provide much value in predicting the damage. Let us drop it for now



In [0]:
#Getting all object columns other than 'has_repair_started'
colNames = object_columns.tolist()
colNames.remove('damage_grade')

In [62]:
test_merged.isnull().sum()

area_assesed                                  0
building_id                                   0
district_id                                   0
has_geotechnical_risk                         0
has_geotechnical_risk_fault_crack             0
has_geotechnical_risk_flood                   0
has_geotechnical_risk_land_settlement         0
has_geotechnical_risk_landslide               0
has_geotechnical_risk_liquefaction            0
has_geotechnical_risk_other                   0
has_geotechnical_risk_rock_fall               0
has_repair_started                        21922
vdcmun_id                                     0
district_id_x                                 0
vdcmun_id_x                                   0
ward_id_x                                     0
count_floors_pre_eq                           0
count_floors_post_eq                          0
age_building                                  0
plinth_area_sq_ft                             0
height_ft_pre_eq                        

In [0]:
#Droping count_families from both test and train

train_merged.drop(['count_families'],inplace=True,axis=1)

In [0]:
test_merged.drop(['count_families'],inplace=True,axis=1)

In [112]:
train_merged.shape
train_copy = train_merged.copy()
train_copy.shape
train_copy.dropna()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Exterior,1f54965821f,Grade 5,21,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Both,2beb42eaeee,Grade 2,30,1.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,Both,25e8adfd09a,Grade 5,26,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Both,392cbd772c9,Grade 3,39,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Exterior,35074b0c35f,Grade 5,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
train_copy.shape

(631761, 57)

In [67]:
train_merged.shape

(631761, 57)

In [0]:
test_merged.shape
test_copy = test_merged.copy()


In [114]:
test_object_columns = test_merged.select_dtypes(include=["object"]).columns
test_object_columns = test_object_columns.tolist()
test_object_columns


['area_assesed',
 'building_id',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'condition_post_eq',
 'legal_ownership_status']

In [0]:
colNames.remove('building_id')

In [0]:
#Converting object columns to categories/codes
# We need to bin continous variables

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


for label in colNames:
  train_copy[label+'_code'] = le.fit_transform(train_copy[label])
  


In [0]:
for label in test_object_columns:
  test_copy[label+'_code'] = le.fit_transform(test_copy[label])

In [0]:
test_copy.drop(columns = test_object_columns,axis=1,inplace=True)
train_copy.drop(columns = colNames,axis = 1,inplace=True)

In [119]:
train_copy.columns

Index(['building_id', 'damage_grade', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_super

In [77]:
train_copy['damage_grade'].head()

0    Grade 4
1    Grade 2
2    Grade 1
3    Grade 5
4    Grade 3
Name: damage_grade, dtype: object

In [0]:
predictor_columns = train_copy.columns.tolist()
predictor_columns.remove('damage_grade')
predictor_columns.remove('building_id')

In [0]:
target_col = 'damage_grade'

In [0]:
from xgboost import XGBClassifier

Now split training data to Train and test split

In [0]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(train_copy[predictor_columns], train_copy[target_col], test_size=0.2)


In [0]:
xgbClass = XGBClassifier()

model = xgbClass.fit(X_train,y_train)

In [127]:
predictions = model.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [128]:
from sklearn.metrics import f1_score
score = f1_score(y_test, predictions, average='weighted')
score

0.7171412059484904

In [129]:
test_predictions = model.predict(test_copy[predictor_columns])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [130]:
test_predictions[:10]

array(['Grade 2', 'Grade 5', 'Grade 5', 'Grade 2', 'Grade 5', 'Grade 3',
       'Grade 3', 'Grade 5', 'Grade 3', 'Grade 3'], dtype=object)

In [0]:
mysubmission = pd.DataFrame({'building_id' : test_merged.building_id, 'damage_grade' : test_predictions })
mysubmission.to_csv('submission.csv', index=False)

In [0]:
from google.colab import files


In [134]:
!ls

a490e594-6-Dataset.zip	datalab  Dataset  submission.csv


In [0]:
files.download('submission.csv')